In [299]:
import pandas as pd
import numpy as np
import math

In [329]:
data_dict = [
    ['Sunny', 'High', 'Weak', 'No'],
    ['Sunny', 'High', 'Strong', 'No'],
    ['Overcast', 'High', 'Weak', 'Yes'],
    ['Sunny', 'High', 'Weak', 'Yes'],
    ['Overcast', 'Normal', 'Strong', 'Yes'],
    ['Sunny', 'Normal', 'Weak', 'Yes'],
    ['Sunny', 'Normal', 'Strong', 'Yes'],
    ['Overcast', 'High', 'Strong', 'Yes'],
    ['Overcast', 'Normal', 'Weak', 'Yes']
]
data = pd.DataFrame(data_dict, columns=['Outlook', 'Humidity', 'Wind', 'Play Golf'])
data.head(2)

,Outlook,Humidity,Wind,Play Golf
0,Sunny,High,Weak,No
1,Sunny,High,Strong,No


In [330]:
def one_hot_encoding(data):
    total_different_values = {}
    for col in data.columns:
        s = set()
        for ele in data[col]:
            s.add(ele)
        temp = []
        temp.append(len(s))
        temp.append(list(s))
        total_different_values[col] = temp

    data_column_wise = []
    for col in data.columns:
        ele = total_different_values[col]
        if ele[0]==1:
            temp = []
            for each_data_item in data[col]:
                temp.append(1)
            data_column_wise.append(temp)
        elif ele[0]==2:
            temp = []
            for each_data_item in data[col]:
                for i in range(len(ele[1])):
                    if each_data_item.lower() == 'no':
                        temp.append(0)
                        break
                    elif each_data_item.lower() == 'yes':
                        temp.append(1)
                        break
                    elif each_data_item == ele[1][i]:
                        temp.append(i)
                        break
            data_column_wise.append(temp)
        else:
            temp = []
            for i in range(len(ele[1])):
                current_category = ele[1][i]
                for each_data_item in data[col]:
                    if each_data_item == current_category:
                        temp.append(1)
                    else:
                        temp.append(0)
                data_column_wise.append(temp)
    arr = np.array(data_column_wise)
    arr = arr.T
    arr = pd.DataFrame(arr)
    return arr

In [331]:
class Node:
    data = []
    left = None
    right = None
    isLeaf = False
    val = -1
    feature_variable_index = -1
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.isLeaf = False
        self.val = -1
        self.feature_variable_index = -1

class DecisionTreeClassifier:
    decision_tree_root = None

    def __init__(self):
        self.decision_tree_root = None

    def entrophy_calculation(self, l):
        if len(l)==0:
            return 0
        # count the positive examples
        total_data = len(l)
        feature_value = 1
        count = 0
        for i in l:
            if i[-1] == feature_value:
                count += 1
        p0 = count / total_data
        p1 = 1-p0
        if p0==0:
            H = - p1 * math.log2(p1)
        elif p1==0:
            H = - p0 * math.log2(p0)
        else:
            H = -p0 * math.log2(p0) - p1 * math.log2(p1)
        return H

    def train(self, l, remaining_features_index):
        # print(f"remaining features: {remaining_features_index}")

        if len(remaining_features_index) == 0:
            needSplitting = False
            output = l[0][-1]
            for i in range(1, len(l)):
                if l[i][-1]!=output:
                    needSplitting = True
                    break
            if needSplitting:
                count_0 = 0
                count_1 = 0
                for i in range(len(l)):
                    if l[i][-1]!=0:
                        count_0 += 1
                    else:
                        count_1 += 1
                newNode = Node(l)
                newNode.isLeaf = True
                if count_0 > count_1:
                    newNode.val = count_0
                else:
                    newNode.val = count_1
                return newNode
            else:
                newNode = Node(l)
                newNode.isLeaf = True
                newNode.val = output
                return newNode

        if len(l)==0:
            return None
        if len(l)==1:
            newNode = Node(l)
            newNode.isLeaf = True
            newNode.val = l[0][-1]
            return newNode
        else:
            # checking whether all nodes have the same output or not
            needSplitting = False
            output = l[0][-1]
            for i in range(1, len(l)):
                if l[i][-1]!=output:
                    needSplitting = True
                    break
            if needSplitting:
                # entrophy of the root node
                root_H = self.entrophy_calculation(l)
                # split the datasets basis on the current feature
                cache = []
                for current_feature in remaining_features_index:
                    negatives_list = []
                    positives_list = []
                    # split the dataset with negative & positive examples
                    for i in l:
                        if i[current_feature] == 0:
                            negatives_list.append(i)
                        else:
                            positives_list.append(i)

                    # print(f"negative list: {negatives_list}")
                    # print(f"positive list: {positives_list}")

                    negative_H = self.entrophy_calculation(negatives_list)
                    positive_H = self.entrophy_calculation(positives_list)

                    IG = root_H - ((len(negatives_list)/len(l))*negative_H + (len(positives_list)/len(l))*positive_H)
                    temp = [IG, current_feature, negatives_list, positives_list]
                    cache.append(temp)

                cache.sort()

                # print(f"current cache state: {cache}")

                splitted_feature = cache[-1][1]
                new_remaining_features_index = []
                for i in remaining_features_index:
                    if i != splitted_feature:
                        new_remaining_features_index.append(i)
                newNode = Node(l)
                newNode.feature_variable_index = splitted_feature
                if len(cache[-1][2])==0:
                    tempNode = Node([])
                    tempNode.isLeaf = True
                    tempNode.val = 0
                    newNode.right = tempNode
                else:
                    newNode.left = self.train(cache[-1][2], new_remaining_features_index)
                if len(cache[-1][3])==0:
                    tempNode = Node([])
                    tempNode.isLeaf = True
                    tempNode.val = 1
                    newNode.right = tempNode
                else:
                    newNode.right = self.train(cache[-1][3], new_remaining_features_index)
                return newNode

            else:
                newNode = Node(l)
                newNode.isLeaf = True
                newNode.val = output
                return newNode

    def fit(self, X, Y):
        l = X.values.tolist()
        for i in range(len(l)):
            l[i].append(Y[i])
        total_features = len(X.columns)
        all_features_index = []
        for i in range(total_features):
            all_features_index.append(i)

        # print(f"all features: {all_features_index}")

        root = self.train(l, all_features_index)
        self.decision_tree_root = root
        return self.decision_tree_root

    def predict(self, l):
        current_node = self.decision_tree_root
        while current_node.isLeaf == False:
            if l[current_node.feature_variable_index] == 0:
                current_node = current_node.left
            else:
                current_node = current_node.right
        return current_node.val

In [332]:
data = one_hot_encoding(data)
X = data.iloc[:, :3]
Y = data.iloc[:, 3]

In [333]:
model = DecisionTreeClassifier()
DT = model.fit(X, Y)

In [334]:
DT.data

[[1, 0, 0, 0],
 [1, 0, 1, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 1],
 [0, 1, 1, 1],
 [1, 1, 0, 1],
 [1, 1, 1, 1],
 [0, 0, 1, 1],
 [0, 1, 0, 1]]

In [335]:
model.predict([1, 1, 0])

1